In [ ]:
import sys, os

from nnunet_ext.run.run_inference import run_inference
from nnunet_ext.run.run_evaluation import run_evaluation

In [ ]:
arch = '3d_fullres'
fold = '0'
device = '4'

trainer = 'nnUNetTrainerLWF'

# Prostate
trained_on = [79, 78, 77, 76]
use_model = [79, 78, 77, 76]
evaluate_on = 79
use_heads = [76, 77] #[79, 78, 77, 76]

# Hyppocampus
#trained_on = [99, 98, 97]
#use_model = [99, 98, 97]
#evaluate_on = 97
#use_head = 98  # 97

In [ ]:
import os

# Adapt checkpoint
from nnunet_ext.scripts.update_checkpoints import modify_checkpoints
# This is always so, repeat for the new_checkpoints iterating over env variables
current_checkpoint_root = '/local/scratch/aranem/Lifelong-nnUNet-storage/'
old_vars = [os.path.join(current_checkpoint_root, var_name) for var_name in ['nnUNet_raw_data_base', 'nnUNet_preprocessed', 'nnUNet_trained_models', 'nnUNet_models_evaluation']]
new_vars = [os.environ['nnUNet_raw_data_base'], os.environ['nnUNet_preprocessed'], os.environ['RESULTS_FOLDER'], os.environ['EVALUATION_FOLDER']]
for current_checkpoint, new_checkpoint in zip(old_vars, new_vars):
    args = [sys.argv[0], arch, trainer, '-trained_on']
    args += [str(x) for x in trained_on]
    args += ['-f', fold, '-r', '-rw', current_checkpoint, new_checkpoint, '-use']
    args += [str(x) for x in use_model]
    sys.argv = args
    modify_checkpoints()

In [ ]:
# Preprocess relevant datasets
import nnunet.experiment_planning.nnUNet_plan_and_preprocess as exp_pp
sys.argv = [sys.argv[0], '-t', str(evaluate_on)]
exp_pp.main()
sys.argv = [sys.argv[0], '-t', str(trained_on[0])]
exp_pp.main()

In [ ]:
for head in use_heads:
    # Extract predictions
    args = [sys.argv[0], arch, trainer, '-trained_on']
    args += [str(x) for x in trained_on]
    args += ['-f', fold, '-use_model']
    args += [str(x) for x in use_model]
    args += ['-evaluate_on', str(evaluate_on), '-d', device, '-use_head', str(head), '--enable_tta']
    # print(args)
    sys.argv = args
    run_inference()